<a href="https://colab.research.google.com/github/sN-Ep/sN-Ep/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to Colab!

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# Load dataset
dataset = load_dataset("json", data_files="chat_pairs.jsonl")["train"]

# Load tokenizer and base model
model_name = "microsoft/DialoGPT-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize
def tokenize(batch):
    texts = [i + tokenizer.eos_token + o + tokenizer.eos_token
             for i, o in zip(batch["input"], batch["output"])]
    tokenized_inputs = tokenizer(texts, truncation=True, padding="max_length", max_length=128)
    tokenized_inputs["labels"] = tokenized_inputs["input_ids"].copy() # Add labels for training
    return tokenized_inputs


dataset = dataset.map(tokenize, batched=True)

# Training settings
args = TrainingArguments(
    output_dir="./my_chat_model",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    warmup_steps=100,
    logging_steps=100,
    save_total_limit=2
)

trainer = Trainer(model=model, args=args, train_dataset=dataset)
trainer.train()

# Save model
trainer.save_model("./my_chat_model")
tokenizer.save_pretrained("./my_chat_model")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("./my_chat_model")
model = AutoModelForCausalLM.from_pretrained("./my_chat_model")

def chat(msg):
    new_input = tokenizer.encode(msg + tokenizer.eos_token, return_tensors='pt')
    reply = model.generate(new_input, max_length=80, pad_token_id=tokenizer.eos_token_id,
                           do_sample=True, top_k=50, top_p=0.95)
    return tokenizer.decode(reply[:, new_input.shape[-1]:][0], skip_special_tokens=True)

print(chat("university"))

Oo ganna giya dekata


In [ ]:
from pyngrok import ngrok

# paste your ngrok token here
ngrok.set_auth_token("2o97l0ylKusEsvAJ1fBUST9dYtr_5EbZKAzfbxQtP5RUNu5mr")

# start a tunnel for Flask
public_url = ngrok.connect(5000)
print("🚀 Public URL:", public_url)


🚀 Public URL: NgrokTunnel: "https://68653b8b8c75.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route("/chat", methods=["POST"])
def chat_api():
    msg = request.json.get("message", "")
    new_input = tokenizer.encode(msg + tokenizer.eos_token, return_tensors='pt')
    reply = model.generate(new_input, max_length=80, pad_token_id=tokenizer.eos_token_id,
                           do_sample=True, top_k=50, top_p=0.95)
    text = tokenizer.decode(reply[:, new_input.shape[-1]:][0], skip_special_tokens=True)
    return jsonify({"reply": text})

app.run(port=5000)
